In [1]:
import os
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
import sys
sys.path.insert(0, '../')

import numpy as np
from TensorRT_Inference import TRTInference

In [2]:
model = TRTInference('wav2vec2-conformer.trt', verbose=True)
model.warmup({'input': np.random.randn(1, 16000)})

In [3]:
import torchaudio
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained('wav2vec2-conformer')

In [4]:
def clean(annotation):
    if "'" in annotation:
        print(annotation, f'has \' in it, removing')
        annotation = annotation.split("'")[0] + annotation.split("'")[1][1:]  # Tokenizer includes "'" but TIL dataset does not, remove the S following '
    annotation = ' '.join(annotation.split())  # Remove extra spaces
    return annotation

In [14]:
audio, sr = torchaudio.load('test/audio/evala_09977.wav')
audio = processor(audio, sampling_rate=16000).input_values[0][0]
output = model({'input': audio})['output'][0]
output = [output, output]
[clean(anno) for anno in processor.batch_decode(np.argmax(output, axis=-1))]

['THE BATTERY LEFT ON THIS PHONE IS ZERO PERCENT',
 'THE BATTERY LEFT ON THIS PHONE IS ZERO PERCENT']

In [15]:
audio.shape

(56064,)